In [1]:
import sys
!{sys.executable} -m pip install mmh3
! {sys.executable} -m pip install datasketch

   ---------------------------------------- 0.0/89.2 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/89.2 kB ? eta -:--:--
   ------------------ --------------------- 41.0/89.2 kB 667.8 kB/s eta 0:00:01
   ---------------------------------------- 89.2/89.2 kB 1.0 MB/s eta 0:00:00


In [5]:
import math
import struct
import numpy as np 
from hashlib import sha1
import mmh3
import pandas as pd
from datasketch import HyperLogLog, HyperLogLogPlusPlus

In [6]:
hashmap_min_size = 4
hashmap_max_size = 16

def get_alpfa(hash_map_size):
    assert hash_map_size >= hashmap_min_size
    assert hash_map_size <= hashmap_max_size

    if hash_map_size == 4:
        return 0.673
    if hash_map_size == 5:
        return 0.697
    if hash_map_size == 6:
        return 0.709

    return 0.7213/((1 + 1.079)/(1 << hash_map_size))

# aka rho in the algorythm
def get_leading_zeros(number, max_bits):
    return max_bits - number.bit_length() + 1 
    


In [7]:
class HLL(object):
    """
    HLL Cardinality counter
    """
    _hash_range_bit = 64
    def __init__(self, p = 8, debug = False):
        self.debug = debug
        
        self.p = p
        self.m = 1 << p;
        self.alpha = get_alpfa(self.p)
        self.M = np.zeros((self.m,), dtype=np.int8)
        self.max_rank = self._hash_range_bit - self.p


    def log(self, text):
        if self.debug:
            print(text)
    
    def getHash(self, value):
        """
        returns hased value
        """
        corected_value = value
        if isinstance(value, str):
            corected_value = value.encode('utf-8')
        elif isinstance(value, float):
            corected_value = str(value).encode('utf-8')
        elif not isinstance(value, bytes):
            corected_value = bytes(value)
        return mmh3.hash64(corected_value)[0]
    
    def add(self, value):
        """
        Adds new value to HLL register
        """
        #hashed value
        x = self.getHash(value)
        self.log(f'hash of {value} is {x}')
        
        # registry index using first p bits of the hash
        j = x & (self.m - 1)
        self.log(f'index = {j}')
        # get the rest bits
        w = x >> self.p
        self.log(f'w = {w}')
        leading_zeroes = get_leading_zeros(w, self.max_rank)
        self.log(f'leading zeores = {leading_zeroes}')
        self.M[j] = max(self.M[j], leading_zeroes)

    def count(self):
        """
        returns estimated cardinality, no bias correction for now
        """
        znamenatel = np.sum(2.0 ** (-self.M))
        self.log(f'znamenatel = {znamenatel}')
        chislitel = float((self.m**2))
        self.log(f'chislitel = {chislitel}')
        
        E = self.alpha * float((self.m**2)) / np.sum(2.0 ** (-self.M))

        self.log(f'E={E}')
        
        if E <= 5*self.m/2:
            registers_wtih_no_data = self.m - np.count_nonzero(self.M)
            if registers_wtih_no_data > 0:
                self.log('linear counting')
                return int(self.m * np.log(self.m / float(registers_with_zeros)))
            else:
                self.log('we cannot do correction')
                return int(E)
        elif E <= (2**32)/30:
            self.log('number is within range')
            return int(E)
        else:
            self.log('We may need some more calculation')
            return int(E)

    def getRegistry(self):
        return self.M

In [34]:
steam_games = pd.read_csv('data/steam-games.csv')

datasets = [
    steam_games['content_descriptor'],
    steam_games['genres'],
    steam_games['categories'],
    steam_games['publisher'],
    steam_games['developer'],
]


for dataset in datasets:
    print('-----------------------------------------------------')
    real_set = set(dataset)
    print(f'Real count {len(real_set)}')
    myHll = HLL(debug=False, p = 12)
    hll = HyperLogLog(p=12)
    hpp = HyperLogLogPlusPlus(p=12)
    
    for item in dataset:
        myHll.add(item)
        if isinstance(item, str):
            hll.update(item.encode('utf-8'))
            hpp.update(item.encode('utf-8'))
    
    print(f'custom implmentation HLL = {myHll.count()}')
    print(f'imported HLL implementation = {hll.count()}')
    print(f'imported HLL plus plus implementation = {hpp.count()}')
    


# hll = HLL(debug=True)
# hll.add("Ivan")
# hll.add("Ivan")
# hll.add("Ivan")
# hll.add("Ivan 1")

# hll.count()

-----------------------------------------------------
Real count 559
custom implmentation HLL = 6519977
imported HLL implementation = 562.9759158094188
imported HLL plus plus implementation = 562.9759158094188
-----------------------------------------------------
Real count 1526
custom implmentation HLL = 7848073
imported HLL implementation = 1528.9303154163078
imported HLL plus plus implementation = 1528.9303154163078
-----------------------------------------------------
Real count 4602
custom implmentation HLL = 13866068
imported HLL implementation = 4678.7643518776185
imported HLL plus plus implementation = 4669.2892845591705
-----------------------------------------------------
Real count 20987
custom implmentation HLL = 78377796
imported HLL implementation = 20975.5390444693
imported HLL plus plus implementation = 21328.666796072182
-----------------------------------------------------
Real count 25127
custom implmentation HLL = 97544284
imported HLL implementation = 25592.3304002

In [41]:
for dataset in datasets:
    print('-----------------------------------------------------')
    real_set = set(dataset)
    real_count = len(real_set)
    print(f'Real count {real_count}')

    errorRates = []
    pValues = range(4,17)
    for p in pValues:
        hll = HyperLogLog(p)
        for item in dataset:
            if isinstance(item, str):
                hll.update(item.encode('utf-8'))
        hllHount = hll.count()
        errorRate = np.abs(hllHount - real_count)* 100 / real_count
        print(f'estiamted count for p={p} is {hllHount} . Error rate: {errorRate}')
        errorRates.append(errorRate)
        

-----------------------------------------------------
Real count 559
estiamted count for p=4 is 678.5496615384616 . Error rate: 21.386343745699747
estiamted count for p=5 is 710.9508482490272 . Error rate: 27.182620438108625
estiamted count for p=6 is 562.7860590461771 . Error rate: 0.6772914214985895
estiamted count for p=7 is 546.9575006341646 . Error rate: 2.1542932675913056
estiamted count for p=8 is 605.983645217694 . Error rate: 8.404945477226109
estiamted count for p=9 is 570.5526505657593 . Error rate: 2.0666637863612407
estiamted count for p=10 is 557.6627059064915 . Error rate: 0.23922971261332554
estiamted count for p=11 is 562.6783363503338 . Error rate: 0.6580208140131985
estiamted count for p=12 is 562.9759158094188 . Error rate: 0.7112550642967406
estiamted count for p=13 is 563.9754074936427 . Error rate: 0.890055007807282


C:\Users\igenchev\anaconda3\Lib\site-packages\datasketch\hyperloglog.py:151: UserWarning: Warning: estimate is close to error correction threshold. Output may not satisfy HyperLogLog accuracy guarantee.
  warnings.warn(


estiamted count for p=14 is 558.4088083086419 . Error rate: 0.10575879988516325
estiamted count for p=15 is 559.7538429928829 . Error rate: 0.13485563378943421
estiamted count for p=16 is 560.3890895812337 . Error rate: 0.2484954528146239
-----------------------------------------------------
Real count 1526
estiamted count for p=4 is 1809.4657641025642 . Error rate: 18.575738145646408
estiamted count for p=5 is 1576.8230248112188 . Error rate: 3.330473447655231
estiamted count for p=6 is 1745.6774499559729 . Error rate: 14.395638922409754
estiamted count for p=7 is 1518.3459940423322 . Error rate: 0.5015731295981506
estiamted count for p=8 is 1584.7991246341862 . Error rate: 3.853153645752702
estiamted count for p=9 is 1557.4908669556637 . Error rate: 2.0636216877892335
estiamted count for p=10 is 1574.7525214690218 . Error rate: 3.1947917083238373
estiamted count for p=11 is 1556.0114343701252 . Error rate: 1.966673287688417
estiamted count for p=12 is 1528.9303154163078 . Error rate:

In [39]:
np.abs(-5)

5

In [30]:
a = 6593149
a.bit_length()

23

In [54]:
2.0**(-np.array([1, 5]))

array([0.5    , 0.03125])

In [37]:
np.array(range(4,16))

array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [7]:
q = 101385018082514226
q.bit_length()

57

In [25]:
1 << 8

256

In [58]:
2 ** 8

256